In [2]:
# import libraries (non-cryptogaphic)
import random # to generate phone numbers
import pandas as pd
from functools import reduce

# import libraries (cryptographic)
import cryptography
import hashlib
import Crypto
import sympy
import secrets

# Generate phone numbers and store them in phone_numbers.csv file

In [3]:
random.seed(10) # to ensure same phone numbers generated every time

In [19]:
# class to generate phone numbers for grab and gojek
class PhoneNumberGenerator:
    def __call__(self, count):
        phone_numbers = random.sample(range(80000000,100000000), count)
        return phone_numbers 
    
# class to store numbers to csv
class PhoneNumberStorageManager:
    def __init__(self):
        self.filename = "phone_numbers.csv"
    def __call__(self, gojek_phone_numbers, grab_phone_numbers):
        d = {"gojek": gojek_phone_numbers, 
            "grab": grab_phone_numbers}
        df = pd.DataFrame(dict([(k,pd.Series(v)) for k,v in d.items()])) # create dataframe
        df.to_csv(self.filename, index = False)

In [20]:
# generate and store phone numbers
gojek_phone_number_count = 61
grab_phone_number_count = 91

# instantiate required classes
phone_number_generator = PhoneNumberGenerator()
phone_number_storage_manager = PhoneNumberStorageManager()

# generate phone numbers
gojek_phone_numbers = phone_number_generator(gojek_phone_number_count)
grab_phone_numbers = phone_number_generator(grab_phone_number_count)

# write phone numbers to csv file
phone_number_storage_manager(gojek_phone_numbers, grab_phone_numbers)


# Define classes for the necessary for the algorithm

In [ ]:
# class to generate clients' private secret
class NumberGenerator:
    
        def generate_safe_prime(size):
            candidate = Crypto.Util.number.getPrime(size)
            print(candidate)
            while True:
                is_safe_prime = sympy.ntheory.isprime((candidate-1)/2) # if same prime, (candidate-1)/2 is prime
                if (is_safe_prime):
                    break
                else:
                    candidate = Crypto.Util.number.getPrime(size)
                    print(candidate)
            return candidate
        
        def generate_random_number(size):
            return secrets.randbits(size)
            
# class to inspect values          
class NumberInspector:
    
    def check_is_primitive_generator(candidate, factors_pminus1, divisor): # note: factors should be the factors of divisor-1
        
        # apply lagrange theorem
        for possible_order in factors_pminus1: # check congruence for all factors (factors is exclusive of p-1 itself)
            result = pow(candidate, possible_order, divisor) # fast modular exponentiation
            if (result == 1):
                return false
        return true
    
    def get_factors(n):
        # retrieved from https://stackoverflow.com/questions/6800193/what-is-the-most-efficient-way-of-finding-all-the-factors-of-a-number-in-python
        return set(reduce(list.__add__, 
                ([i, n//i] for i in range(1, int(n**0.5) + 1) if n % i == 0)))
                
    

    
        

# Create psi client class

In [ ]:
# set global parameters
set_size = 1000 # pre-agreed set size to exchange

In [ ]:
# client class (both grab and gojek are clients communicating directly with each other)
class psi_client:
    def __init__(private_key_size, phone_numbers, p, factors_pminus1)
        self.number_inspector = NumberInspector()
        self.private_key = generate_random_number(private_key_size)
        self.set = phone_numbers
        self.p = p
        self.factors_pminus1 = factors_pminus1

    def hash_to_primitive_root_modulo_p(element):
        hash_hex = hashlib.sha256(element).hexdigest() #sha3_256
        hash_int = int(hash_hex, 16)
        while True:
            is_primitive_generator = self.number_inspector.check_is_primitive_generator(
                hash_int, self.factors_pminus1, self.p
            )
            if (is_primitive_generator):
                break
            else:
                hash_hex = hashlib.sha256(hash_int).hexdigest()
                hash_int = int(hash_hex, 16)
        return hash_int
    
    def modular_exponentation(element):
        return pow(element, self.private_key, self.p)
        
    def encrypt_set(is_other_party, other_party_set=None):
        if (is_other_party):
            set_to_encrypt = other_party_set
        else:
            set_to_encrypt = self.set
        
        encrypted_values = []
        for element in set_to_encrypt:
            hashed_value = hash_to_primitive_root_modulo_p(element)
            encrypted_value = modular_exponentation(hashed_value)
            encrypted_values.append(encrypted_value)
        
        return encrypted_values
        
    def get_intersection(my_set, other_party_set):
        
        encrypted_common_values = set(my_set).intersection(other_party_set)
        index_of_common_values = []
        
        for element in encrypted_common_values:
            index_of_common_values.append(my_set.index(element))
            
        common_values = []
        
        for index in index_of_common_values:
            common_values.append(my_set[index])
            
        return common_values
        
            
        

# Algorithm